To set up and begin running DeepGlycanSite, we first need to create the environment that contains all of its dependencies. I have already established the environment, we just need to run it and make sure its working. 

In [20]:
from pathlib import Path

import docker
import docker.errors
from docker.models.containers import Container

root = Path().resolve() #Root of this local project directory   

try:
    client: docker.client.DockerClient = docker.from_env()
except Exception as e:
    print(e)

client

In [28]:
project_name = "docking_simulation_helpers" #Do Not Change
service_name = "deepglycansite" #Here, choose the docking service you want to use as named in 'docker-compose.yml'

containers = client.containers.list(all=True)

app_container: Container = None

for container in containers:
    if f"{project_name}-{service_name}-1" in container.name:
        print(f"Starting container: {container.name}")
        try:
            container.start()
            print(container.name)
            app_container = container
            break
        except docker.errors.APIError as e:
            print(e)
    

Starting container: docking_simulation_helpers-deepglycansite-1
docking_simulation_helpers-deepglycansite-1


In [31]:
in_folder = "User" #Change to your name
ligand_sdf = Path(str(root) + "/folder/example.sdf").resolve() #Change /folder/example.sdf to the place where your file is in this directory.
print(ligand_sdf)
receptor_pdb = Path(str(root) + "/folder/receptor.pdb").resolve() #Change /folder/receptor.pdb the same way
print(receptor_pdb)
# Ensure the input folder exists in the docker container

cmd = f"mkdir -p {in_folder}"

app_container.exec_run(cmd)

exit_code, output = app_container.exec_run(cmd)

exit_code, output

/Users/higgins/Documents/GitHub/docking_simulation_helpers/folder/example.sdf
/Users/higgins/Documents/GitHub/docking_simulation_helpers/folder/receptor.pdb


(0, b'')

In [24]:
import torch

data = torch.load('Janique/Janique_af.txt.pt')

FileNotFoundError: [Errno 2] No such file or directory: 'Janique/Janique_af.txt.pt'

In [3]:
data

[[tensor([[0.9406]]),
  Data(x=[712, 1309], edge_index=[2, 7038], pos=[712, 3], edge_feats=[7038, 9], protein_name=[712], pos_CA=[712, 3], pos_CB=[712, 3], pos_N=[712, 3])]]